# 10. Convolutional Neural Networks

Please work on Google Colab. (No.)

In [ ]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

from IPython.display import display

In [ ]:
from hw_utils import *
from hw_models import *

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib

import tensorflow.keras.callbacks as kc

import sklearn
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split

In [ ]:
print(tf.__version__)

In [ ]:
tf.config.list_physical_devices()

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
device_lib.list_local_devices()

In [ ]:
data = './data/'
out = './out/'

# Bold print for Jupyter Notebook
b1 = '\033[1m'
b0 = '\033[0m'

### Just some matplotlib and seaborn parameter tuning

In [ ]:
axistitlesize = 20
axisticksize = 17
axislabelsize = 26
axislegendsize = 23
axistextsize = 20
axiscbarfontsize = 15

# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

mpl.rcParams.update({'figure.autolayout': False})

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

# Colorpalettes, colormaps, etc.
sns.set_palette(palette='rocket')

## 1. Load the MNIST dataset and create a CNN model

- load the MNIST dataset from the tensorflow/keras built-in dataset (just like last time)
- use the original train/test split!
- divide each pixel's value by 255 and now do not reshape, leave it as is (2D matrix (28x28) )
- eg for the test set you will have a (10000, 28, 28) shaped vector
- train the following network on the training set and generate prediction for the 10.000 test images:

        input (28, 28)
        conv2D, 16 kernels, kernel size = 3, valid padding, relu activation
        conv2D, 16 kernels, kernel size = 3, valid padding, relu activation
        maxpooling kernel size = 2*2
        conv2D, 32 kernels, kernel size = 3, valid padding, relu activation
        conv2D, 32 kernels, kernel size = 3, valid padding, relu activation
        maxpooling kernel size = 2*2
        flatten
        dense, 10 neurons, softmax activation
    * pay attention to channel format, you will need to expand dims!
    * how many parameters do we have for each layer?
    * use Adam optimizer with default parameters
    * use categorical crossentropy as loss function
    * compile the model
    * print out a summary of the model
    * train the CNN on the training data for 5 epochs with batch size of 32
    * use the test data as validation data
        
- calculate the categorical cross-entropy loss and the accuracy! Hint: you should get at least ~98% accuracy
- show the confusion matrix of the predictions (predicted values vs actual labels)
- where does the model make mistakes? Where does it improve compared to fully connected nets?

### 1./a. Load and preprocess the MNIST dataset

In [ ]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')

In [ ]:
print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)
print('X_test shape :', X_test.shape)
print('y_test shape :', y_test.shape)

#### Look at some random images

In [ ]:
nrows = 3
ncols = 8
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*2, nrows*2),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
fig.subplots_adjust(hspace=0.5)

rand_idx = np.random.randint(0, len(X_train), size=nrows*ncols)
images = X_train[rand_idx]
labels = y_train[rand_idx]

for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(images[i], cmap='Greys_r')
    ax.set_title('Label : {0}'.format(labels[i]), fontweight='bold',
                 color='white', pad=0)
    ax.axis('off')
    ax.grid(False)

plt.suptitle('Fig. 1. Sample data along with their labels of the MNIST dataset.',
             color='white', fontsize=axistitlesize+5, y=0.1)
    
plt.show()

#### Convert labels to one-hot encoded arrays and scale data

In [ ]:
# Scale data
X_train = X_train / 255
X_test = X_test / 255
# Reshape data
X_train = X_train.reshape((-1, 28,28, 1))
X_test = X_test.reshape((-1, 28,28, 1))
# Convert labels to one-hot encoded arrays
y_train = label_binarize(y_train, classes=np.unique(y_train))
y_test = label_binarize(y_test, classes=np.unique(y_test))

In [ ]:
print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)
print('X_test shape :', X_test.shape)
print('y_test shape :', y_test.shape)

### 1./b. Define model for task 1.

In [ ]:
with tf.device('/device:GPU:0'):
    # Create checkpoint file to save best model into
    best_model_ex1 = kc.ModelCheckpoint('./models/best_model_ex1.hdf5', save_best_only=True, verbose=1)
    
    # Define the model
    cnn_model_ex1 = cnn_model(imsize=28, stride=1, kernelsize=3,
                              n_channels=1, num_of_filters=16, reg=5e-5,
                              padding='valid', activation='relu', n_class=10,
                              model_name='model_ex1')
    
    # Configure the model
    cnn_model_ex1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                          loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0),
                          metrics=['accuracy'])

In [ ]:
cnn_model_ex1.summary()

### 1./c. Fitting the model

In [ ]:
epochs_ex1 = 5

In [ ]:
with tf.device('/device:GPU:0'):
    history_ex1 = cnn_model_ex1.fit(x=X_train,
                                    y=y_train,
                                    batch_size=32,
                                    epochs=epochs_ex1,
                                    verbose=1,
                                    validation_split=0.2,
                                    shuffle=True,
                                    callbacks=[best_model_ex1],
                                    initial_epoch=0,
                                    steps_per_epoch=None)

### 1./d. Evaluate loss and accuracy history

In [ ]:
nrows = 1
ncols = 2
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*12, nrows*8),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})

# LOSS GRAPH
ax = axes[0]
ax.plot(history_ex1.epoch, history_ex1.history['loss'], label='Training loss',
        c=cm.magma(0.75), lw=5)
ax.plot(history_ex1.epoch, history_ex1.history['val_loss'], label='Validation loss',
        c=cm.magma(0.93), lw=5)
ax.set_ylabel('Loss', fontsize=axislabelsize, fontweight='bold',
              color='white')

# ACCURACY GRAPH
ax = axes[1]
ax.plot(history_ex1.epoch, history_ex1.history['accuracy'], label='Training accuracy',
        c=cm.magma(0.75), lw=5)
ax.plot(history_ex1.epoch, history_ex1.history['val_accuracy'], label='Validation accuracy',
        c=cm.magma(0.93), lw=5)
ax.set_ylabel('Accuracy', fontsize=axislabelsize, fontweight='bold',
              color='white')

for ax in axes.reshape(-1):
    ax.set_xlabel('Epochs', fontsize=axislabelsize, fontweight='bold',
                  color='white')
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
    ax.tick_params(axis='both', which='major', labelsize=axisticksize,
                   colors='white')

    ax.legend(fontsize=axislegendsize)

plt.suptitle('Fig. 2. The loss and accuracy history of the CNN model trained on MNIST dataset. The loss history can be\n' +
             'seen on the left side, while the training accuracy on the right.',
             color='white',
             fontsize=axistitlesize+5, y=-0.02)
    
plt.show()

### 1./e. Make predictions with the model

In [ ]:
y_pred = cnn_model_ex1.predict(X_test)

In [ ]:
# Convert one-hot encoded preds and tests to normal arrays
y_test_b = y_test.argmax(axis=-1)
y_pred_b = y_pred.argmax(axis=-1)

# Calculate and accuracy metric and the confusion matrix
accuracy = accuracy_metric(y_test=y_test_b, y_pred=y_pred_b)
conf_mat = confusion_matrix(y_test_b, y_pred_b, labels=[i for i in range(0,10)])

In [ ]:
plot_confusion_matrix(conf_mat, y_test_b, labels=[i for i in range(0,10)],
                      title=('Fig. 3. Confusion matrix of the predictions\n' +
                             'on the test set of my CNN model.\n' +
                             'Accuracy of model is {0:.3f}%'.format(accuracy)))

In [ ]:
nrows = 3
ncols = 8
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*2, nrows*2),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
fig.subplots_adjust(hspace=0.5)

rand_idx = np.random.randint(0, len(X_test), size=nrows*ncols)
images = X_test[rand_idx]
labels_gr = y_test_b[rand_idx]
labels_tr = y_pred_b[rand_idx]

for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(images[i].reshape((28,28)), cmap='Greys_r')
    ax.set_title('True label : {0}\nPred label : {1}'.format(labels_gr[i],
                                                             labels_tr[i]),
                 fontweight='bold', color='white', pad=0)
    ax.axis('off')
    ax.grid(False)

plt.suptitle('Fig. 4. Predictions of the basic CNN model on the test set of the MNIST dataset\n' +
             'by noting groundtruth and predicted labels along with the\n' +
             'corresponding images.',
             color='white', fontsize=axistitlesize+5, y=0.1)
    
plt.show()

## 2. Download the Street View House Numbers (SVHN) Dataset

- source: http://ufldl.stanford.edu/housenumbers/
- use the cropped dataset!
- to get the dataset use eg. wget and keep the original splitting, so download train and test matrix files
- preprocess the downloaded data to be able to use it for training and testing, so shapes should be same (except image sizes) as it was in ex 1.
- how many classes do we have in the dataset? how many train and test examples do we have?
- what is the dimension of the images?
- show 5 images from the dataset
- make one-hot encoding for the labels

### 2./a. Load and preprocess the SVHN dataset

The dataset at http://ufldl.stanford.edu/housenumbers/ isn't reachable for me at the moment I'm solving this assignment [at 2020-11-22T15:52:43.164052]. So I'm using another method to obtain and load the SVHN dataset.

Sources:
- Loading data:
    - https://www.machinecurve.com/index.php/2020/01/10/making-more-datasets-available-for-keras/
    - https://github.com/christianversloot/extra_keras_datasets/blob/master/extra_keras_datasets/svhn.py

This can reach the Stanford dataset (somehow...)

In [ ]:
from extra_keras_datasets import svhn

In [ ]:
(X_train, y_train), (X_test, y_test) = svhn.load_data(type='normal')
# Define SVHN classes in order
svhn_classes = [1,2,3,4,5,6,7,8,9,0]

In [ ]:
print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)
print('X_test shape :', X_test.shape)
print('y_test shape :', y_test.shape)

In [ ]:
nrows = 3
ncols = 8
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*2, nrows*2),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
fig.subplots_adjust(hspace=0.5)

rand_idx = np.random.randint(0, len(X_train), size=nrows*ncols)
images = X_train[rand_idx]
labels = y_train[rand_idx]

for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(images[i], cmap='Greys_r')
    ax.set_title('Label : {0}'.format(svhn_classes[labels[i]-1]), fontweight='bold',
                 color='white', pad=0)
    ax.axis('off')
    ax.grid(False)

plt.suptitle('Fig. 5. Sample data along with their labels of the SVHN dataset.',
             color='white', fontsize=axistitlesize+5, y=0.1)
    
plt.show()

### 2./c. Preprocess the dataset

In [ ]:
# Scale data
X_train = X_train / 255
X_test = X_test / 255
# Reshape data
X_train = X_train.reshape((-1, 32,32, 3))
X_test = X_test.reshape((-1, 32,32, 3))
# Convert labels to one-hot encoded arrays
y_train = label_binarize(y_train, classes=np.unique(y_train))
y_test = label_binarize(y_test, classes=np.unique(y_test))

In [ ]:
print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)
print('X_test shape :', X_test.shape)
print('y_test shape :', y_test.shape)

## 3. Train the CNN model seen in the 1st exercise for this dataset
* create a convolutional neural network
* the network should have the following layers:
        
        input (32, 32, 3)
        conv2D, 16 kernels, kernel size = 3, valid padding, relu actvation
        conv2D, 16 kernels, kernel size = 3, valid padding, relu actvation
        maxpooling kernel size = 2*2
        conv2D, 32 kernels, kernel size = 3, valid padding, relu actvation
        conv2D, 32 kernels, kernel size = 3, valid padding, relu actvation
        maxpooling kernel size = 2*2
        flatten
        dense, 10 neurons, softmax activation
        how many parameters do we have for each layer?

    * use Adam optimizer with default parameters
    * use categorical crossentropy as loss function
    * compile the model
    * print out a summary of the model
    * train the CNN on the training data for 15 epochs with batch size of 32
    * use the test data as validation data
- calculate the categorical cross-entropy loss and the accuracy! Hint: you should get at least ~80-90% accuracy

### 3./a. Define model for task 3.

In [ ]:
with tf.device('/device:GPU:0'):
    # Create checkpoint file to save best model into
    best_model_ex3 = kc.ModelCheckpoint('./models/best_model_ex3.hdf5', save_best_only=True, verbose=1)
    
    # Define the model
    cnn_model_ex3 = cnn_model(imsize=32, stride=1, kernelsize=3,
                              n_channels=3, num_of_filters=16, reg=5e-5,
                              padding='valid', activation='relu', n_class=10,
                              model_name='model_ex3')
    
    # Configure the model
    cnn_model_ex3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                          loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0),
                          metrics=['accuracy'])

In [ ]:
cnn_model_ex3.summary()

In [ ]:
epochs_ex3 = 15

In [ ]:
with tf.device('/device:GPU:0'):
    history_ex3 = cnn_model_ex3.fit(x=X_train,
                                    y=y_train,
                                    batch_size=32,
                                    epochs=epochs_ex3,
                                    verbose=1,
                                    validation_split=0.2,
                                    shuffle=True,
                                    callbacks=[best_model_ex3],
                                    initial_epoch=0,
                                    steps_per_epoch=None)

## 4. Evaluate performance

- plot the training and the validation loss on the same plot!
- plot the training and the validation accuracy on the same plot!
- do we overfit?
- show the confusion matrix of the predictions (predicted values vs actual labels)
- where does the model make mistakes?

### 4./a. Evaluate loss and accuracy history

In [ ]:
nrows = 1
ncols = 2
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*12, nrows*8),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})

# LOSS GRAPH
ax = axes[0]
ax.plot(history_ex3.epoch, history_ex3.history['loss'], label='Training loss',
        c=cm.magma(0.75), lw=5)
ax.plot(history_ex3.epoch, history_ex3.history['val_loss'], label='Validation loss',
        c=cm.magma(0.93), lw=5)
ax.set_ylabel('Loss', fontsize=axislabelsize, fontweight='bold',
              color='white')

# ACCURACY GRAPH
ax = axes[1]
ax.plot(history_ex3.epoch, history_ex3.history['accuracy'], label='Training accuracy',
        c=cm.magma(0.75), lw=5)
ax.plot(history_ex3.epoch, history_ex3.history['val_accuracy'], label='Validation accuracy',
        c=cm.magma(0.93), lw=5)
ax.set_ylabel('Accuracy', fontsize=axislabelsize, fontweight='bold',
              color='white')

for ax in axes.reshape(-1):
    ax.set_xlabel('Epochs', fontsize=axislabelsize, fontweight='bold',
                  color='white')
    ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
    ax.tick_params(axis='both', which='major', labelsize=axisticksize,
                   colors='white')

    ax.legend(fontsize=axislegendsize)

plt.suptitle('Fig. 6. The loss and accuracy history of the CNN model trained on SVHN dataset. The loss history can be\n' +
             'seen on the left side, while the training accuracy on the right.',
             color='white',
             fontsize=axistitlesize+5, y=-0.02)
    
plt.show()

Overfit happens, when the validation loss starting to stagnate (or grow), while the training loss further decreases. That's exactly the case here, as seen on the graph.

### 4./b. Make predictions with the model

In [ ]:
y_pred = cnn_model_ex3.predict(X_test)

In [ ]:
# Convert one-hot encoded preds and tests to normal arrays
y_test_b = y_test.argmax(axis=-1)
y_pred_b = y_pred.argmax(axis=-1)

# Calculate and accuracy metric and the confusion matrix
accuracy = accuracy_metric(y_test=y_test_b, y_pred=y_pred_b)
conf_mat = confusion_matrix(y_test_b, y_pred_b, labels=svhn_classes)

In [ ]:
plot_confusion_matrix(conf_mat, y_test_b, labels=svhn_classes,
                      title=('Fig. 7. Confusion matrix of the predictions\n' +
                             'on the SVHN test set of my CNN model.\n' +
                             'Accuracy of model is {0:.3f}%'.format(accuracy)))

In [ ]:
nrows = 3
ncols = 8
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*2, nrows*2),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
fig.subplots_adjust(hspace=0.5)

rand_idx = np.random.randint(0, len(X_test), size=nrows*ncols)
images = X_test[rand_idx]
labels_gr = y_test_b[rand_idx]
labels_tr = y_pred_b[rand_idx]

for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(images[i])
    ax.set_title('True label : {0}\nPred label : {1}'.format(svhn_classes[labels_gr[i]],
                                                             svhn_classes[labels_tr[i]]),
                 fontweight='bold', color='white', pad=0)
    ax.axis('off')
    ax.grid(False)

plt.suptitle('Fig. 8. Predictions of the basic CNN model on the test set of the SVHN dataset\n' +
             'by noting groundtruth and predicted labels along with the corresponding images.',
             color='white', fontsize=axistitlesize+5, y=0.1)
    
plt.show()

## 5. Train an other CNN
- as we can see the previous architecture can be further improved
- come up with an architecture that can achieve more than 91% accuracy on the test set
- print out the summary for this model!
- plot the loss and accuracy curves for this model too!

### 5./a. Create and fit better model

- Added another convolutional block to the network
- Changed initial number of filters from `16` to `32`
- Changed learning rate from `0.005` to `0.004`
- Changed padding from `valid` to `same`
- Added early stopping with `patience=10` epochs

In [ ]:
with tf.device('/device:GPU:0'):
    # Create checkpoint file to save best model into
    best_model_ex5 = kc.ModelCheckpoint('./models/best_model_ex5.hdf5', save_best_only=True, verbose=1)
    # Configure early stopping with N epochs of patience
    es_ex5 = kc.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    
    # Define the model
    cnn_model_ex5 = better_cnn_model(imsize=32, stride=1, kernelsize=3,
                                     n_channels=3, num_of_filters=32,
                                     padding='same', activation='relu', n_class=10,
                                     model_name='model_ex5')
    
    # Configure the model
    cnn_model_ex5.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.004),
                          loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0),
                          metrics=['accuracy'])

In [ ]:
cnn_model_ex5.summary()

In [ ]:
epochs_ex5 = 100

In [ ]:
with tf.device('/device:GPU:0'):
    history_ex5 = cnn_model_ex5.fit(x=X_train,
                                    y=y_train,
                                    batch_size=32,
                                    epochs=epochs_ex5,
                                    verbose=1,
                                    validation_split=0.2,
                                    shuffle=True,
                                    callbacks=[es_ex5, best_model_ex5],
                                    initial_epoch=0,
                                    steps_per_epoch=None)

### 5./b. Evaluate loss and accuracy history

In [ ]:
nrows = 1
ncols = 2
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*12, nrows*8),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})

# LOSS GRAPH
ax = axes[0]
ax.plot(history_ex5.epoch, history_ex5.history['loss'], label='Training loss',
        c=cm.magma(0.75), lw=5)
ax.plot(history_ex5.epoch, history_ex5.history['val_loss'], label='Validation loss',
        c=cm.magma(0.93), lw=5)
ax.set_ylabel('Loss', fontsize=axislabelsize, fontweight='bold',
              color='white')

# ACCURACY GRAPH
ax = axes[1]
ax.plot(history_ex5.epoch, history_ex5.history['accuracy'], label='Training accuracy',
        c=cm.magma(0.75), lw=5)
ax.plot(history_ex5.epoch, history_ex5.history['val_accuracy'], label='Validation accuracy',
        c=cm.magma(0.93), lw=5)
ax.set_ylabel('Accuracy', fontsize=axislabelsize, fontweight='bold',
              color='white')

for ax in axes.reshape(-1):
    if epochs_ex5 > history_ex5.epoch[-1] : ax.axvline(x=history_ex5.epoch[-1], label='Early stopping',
                                                       color=cm.magma(0.5), ls='--', lw=4)
    ax.set_xlabel('Epochs', fontsize=axislabelsize, fontweight='bold',
                  color='white')
    #ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
    ax.tick_params(axis='both', which='major', labelsize=axisticksize,
                   colors='white')

axes[0].legend(loc='upper right', fontsize=axislegendsize)
axes[1].legend(loc='lower right', fontsize=axislegendsize)

plt.suptitle('Fig. 9. The loss and accuracy history of the BETTER CNN model trained on SVHN dataset. The loss history can be\n' +
             'seen on the left side, while the training accuracy on the right.',
             color='white',
             fontsize=axistitlesize+5, y=-0.02)
    
plt.show()

### 5./c. Make predictions with the model

In [ ]:
y_pred = cnn_model_ex5.predict(X_test)

In [ ]:
# Convert one-hot encoded preds and tests to normal arrays
y_test_b = y_test.argmax(axis=-1)
y_pred_b = y_pred.argmax(axis=-1)

# Calculate and accuracy metric and the confusion matrix
accuracy = accuracy_metric(y_test=y_test_b, y_pred=y_pred_b)
conf_mat = confusion_matrix(y_test_b, y_pred_b, labels=svhn_classes)

In [ ]:
plot_confusion_matrix(conf_mat, y_test_b, labels=svhn_classes,
                      title=('Fig. 10. Confusion matrix of the predictions\n' +
                             'on the SVHN test set of my BETTER CNN model.\n' +
                             'Accuracy of model is {0:.3f}%'.format(accuracy)))

In [ ]:
nrows = 3
ncols = 8
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*2, nrows*2),
                         facecolor='black', subplot_kw={'facecolor' : 'black'})
fig.subplots_adjust(hspace=0.5)

rand_idx = np.random.randint(0, len(X_test), size=nrows*ncols)
images = X_test[rand_idx]
labels_gr = y_test_b[rand_idx]
labels_tr = y_pred_b[rand_idx]

for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(images[i])
    ax.set_title('True label : {0}\nPred label : {1}'.format(svhn_classes[labels_gr[i]],
                                                             svhn_classes[labels_tr[i]]),
                 fontweight='bold', color='white', pad=0)
    ax.axis('off')
    ax.grid(False)

plt.suptitle('Fig. 11. Predictions of the BETTER CNN model on the test set of the SVHN dataset\n' +
             'by noting groundtruth and predicted labels along with the\n' +
             'corresponding images.',
             color='white', fontsize=axistitlesize+5, y=0.1)
    
plt.show()